## StreamStats API Scraper Automatic

__Description__: Tool to automatically run the [USGS StreamStats tool](https://www.usgs.gov/mission-areas/water-resources/science/streamstats-streamflow-statistics-and-spatial-analysis-tools?qt-science_center_objects=0#qt-science_center_objects) for multiple points within a catchment and return the flow frequency curves and subcatchment boundaries.

__Input__: A shapefile containing the latitude and longitude of points on the stream grid for the specified state (confluence and main stem locations).

__Output__: GeoJSON file containing the delinated catchment boundary and flow frequency data for each point, as well as a CSV file containing the flow frequency data.

*Authors*: sputnam@Dewberry.com & slawler@Dewberry.com

### Load libraries and Python options:

In [1]:
import os
import re
import sys
import json
sys.path.append('../USGStools')
from StreamStats_API_Scraper import*
import geopandas as gpd
from geojson import dump

### Specify the state abbreviation and location of the shapefile: 

##### Specify:

In [2]:
#The state abbreviation in uppercase
state='NY' 

#Specify the location of the shapefile containing the lat/lon of points on the stream grid
path=r'C:\Users\tmiesse\Desktop\bridges' 
name='Bridges_StreamStatSnap.shp' #The name of the shapefile

#Specify a consistent coordinate reference system
use_epsg='4326' 

#Location to save the StreamStats results for each polygon
allresults=os.path.join(path,'AllStreamStats') 
if os.path.isdir(allresults)==False: 
    os.mkdir(allresults)    

##### Load the shapefile:

In [21]:
#Read the shapefile as a geopandas dataframe
#Transform the coordinate reference system of the geodataframe
gdf=gpd.read_file(os.path.join(path, name)) 
gdf=gdf.to_crs({'init': 'epsg:{0}'.format(use_epsg)}) 

In [22]:
gdf.index = gdf['Id'].values

##### Initialize parameters

In [17]:
#Dictionary to store the catchment polygons (catchment boundaries) 
polyg={} 

#Dictionary to store the outlet flow frequency data dictionaries
ffdata={} 

get_flow=True
print_status=True
if state=='WI': get_flow=False 
errors = [0]
count = {i:0 for i in gdf['Id']}
keys = count.keys()

### Run the API tool for each point:

In [25]:

#while errors != [] and [count[key] > 3 for key in keys]:
    errors = []
    for i in gdf.index.values:
        try:
            #Longitude and latitude for each shapely point and the confluence number
            lon, lat, ID_Num = gdf.geometry[i].x, gdf.geometry[i].y, gdf['Id'][i] 
            if print_status: print("Lat/Lon/Confluence:", lat, lon, ID_Num)
                
            #Run the SS_scrape function. Option: set status=False to hide print statements
            polyg[ID_Num], ff_json  = SS_scrape(state, lon, lat, use_epsg, print_status) 
            if get_flow: 
                #Use the function above to extract the json data
                ffdata[ID_Num]                           = get_peaks(ff_json) 
                polyg[ID_Num]['features'][0]['ffcurve']  = ffdata[ID_Num]
            
            with open(os.path.join(allresults,'StreamStats_Polygons_{0}.geojson'.format(int(ID_Num))), 'w') as f:
                dump(polyg[ID_Num], f)
        except:
            print('could not process data {}'.format(gdf['Id'][i]))
            errors.append(gdf['Id'][i]) 
            count[gdf['Id'][i]] += 1 

Lat/Lon/Confluence: 44.49430728613341 -74.87051208078036 122
Fetched Peak Flows


### Load the results:

In [26]:
files=load_files(allresults)
gdf2, ffdic=load_results(files, use_epsg)

337 Polygon Files Found


In [24]:
gdf.drop(gdf.loc[gdf['Id'][gdf2['ID_Num']]].index, inplace=True)
gdf.index.values

array([122], dtype=int64)

###  Save:

##### The flow frequency data as a CSV:

In [27]:
#Run this function to construct the summary table for all outlet locations
if get_flow: ff_df=ff_summary(ffdic) 
    
#Save the results as a csv
if get_flow: ff_df.to_csv(os.path.join(path,'StreamStats_FlowFrequency.csv')) 

                0           1          10         100          101  \
RI                                                                   
1.25  2722.476473  105.367471  371.134293  138.411019   587.438697   
1.5   3131.739667  127.123751  440.892592  167.686939   698.958181   
2.0   3631.339930  155.821593  531.378270  206.309984   842.119320   
5.0   4949.987856  234.758003  777.472063  312.073232  1227.785959   
10.0  5852.257050  292.375717  957.029999  388.026610  1500.246808   

             102          103          104          105         106  ...  \
RI                                                                   ...   
1.25  222.689392   630.967702   521.398561   423.581278  302.410724  ...   
1.5   268.155301   758.453009   626.716159   509.534927  364.508898  ...   
2.0   327.444645   923.917443   761.759774   620.265057  445.150642  ...   
5.0   489.072720  1373.053460  1128.332168   921.317926  665.009716  ...   
10.0  604.588797  1693.643605  1386.394517  1134.3045

##### The catchment polygons as a Shapefile:

In [28]:
#Export the geodataframe as a shapefile
gdf2 = convert_attr(gdf2)
gdf2.to_file(filename = os.path.join(path,'StreamStats_Polygons.shp')) 

C:\Users\tmiesse\AppData\Local\Continuum\anaconda3\envs\geoprocess\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


##### The catchment polygons as a geojson:

In [29]:
with open(os.path.join(path,'StreamStats_Polygons.geojson'), 'w') as f:
     dump(gdf2, f)  

# END